## 1、review

### 1） 当 tensor 连续时

In [32]:
import torch

a = torch.tensor([1, 2, 3, 4, 5, 6])
b = a.view(2, 3)

print(a)
print(b)
print(b.is_contiguous())

# 查看结果，会发现二者输出一致，表示存储区的数据并没有发生改变
print(a.storage())
print(b.storage())

# 查看结果，会发现二者输出一致，表示 a 和 b 共享存储区
print(a.storage().data_ptr())
print(b.storage().data_ptr())

# 查看结果，会发现二者在元数据 (metadata) 中的 stride 信息发生了改变
print(a.stride())
print(b.stride())

tensor([1, 2, 3, 4, 5, 6])
tensor([[1, 2, 3],
        [4, 5, 6]])
True
 1
 2
 3
 4
 5
 6
[torch.storage.TypedStorage(dtype=torch.int64, device=cpu) of size 6]
 1
 2
 3
 4
 5
 6
[torch.storage.TypedStorage(dtype=torch.int64, device=cpu) of size 6]
140203173773632
140203173773632
(1,)
(3, 1)


### 2） 当 tensor 不连续时
不连续的 tensor 是不能使用 torch.view() 方法的，否则会报错

In [13]:
import torch

a = torch.tensor([1, 2, 3, 4, 5, 6]).view(2, 3)
b = a.t()

print(a)
print(b)
print(b.is_contiguous())

c = b.view(6, 1)
print(c)

tensor([[1, 2, 3],
        [4, 5, 6]])
tensor([[1, 4],
        [2, 5],
        [3, 6]])
False


RuntimeError: view size is not compatible with input tensor's size and stride (at least one dimension spans across two contiguous subspaces). Use .reshape(...) instead.

如果一定要用 torch.view() 方法，就必须先使用 .contiguous() 方法，让 tensor 先变得连续（重新开辟一块内存空间，生成一个新的、连续的张量对象），再使用 .view() 方法

In [15]:
import torch

a = torch.tensor([1, 2, 3, 4, 5, 6]).view(2, 3)
b = a.t()

b = b.contiguous()
print(b.is_contiguous())

c = b.view(6, 1)
print(c)

True
tensor([[1],
        [4],
        [2],
        [5],
        [3],
        [6]])


## 2、tensor.reshape()

### 1）当 tensor 连续时
当 tensor 连续时，  tensor.reshape() 与  tensor.view() 效果一样，会和原来 tensor 共用存储区

In [19]:
import torch

a = torch.tensor([1, 2, 3, 4, 5, 6]).view(2, 3)

print(a)
print(b)
print(b.is_contiguous())

c = b.view(6, 1)
print(c)

tensor([[1, 2, 3],
        [4, 5, 6]])
tensor([[1, 2, 3],
        [4, 5, 6]])
True
tensor([[1],
        [2],
        [3],
        [4],
        [5],
        [6]])


### 2）当 tensor 不连续时， reshape() = contiguous() + view()，会先通过产生新的存储区的 tensor，与原来 tensor 不共用存储区

In [33]:
import torch

a = torch.tensor([1, 2, 3, 4, 5, 6]).view(2, 3)
b = a.t()

print(a)
print(b)
print(b.is_contiguous())

c = b.reshape(6, 1)
print(c)

# 查看结果，会发现二者输出不一致，表示 a 和 b 不共享存储区
print(b.storage().data_ptr())
print(c.storage().data_ptr())

tensor([[1, 2, 3],
        [4, 5, 6]])
tensor([[1, 4],
        [2, 5],
        [3, 6]])
False
tensor([[1],
        [4],
        [2],
        [5],
        [3],
        [6]])
140203159189120
140203170387328


## 3、tensor.resize()

### 1）当原始数据 元素多余的时候
由以下代码，我们可以观察到，存储区的地址始终都没有改变。a的原始数据始终是 1～7，tensor a 只取了前6个

In [49]:
import torch

a = torch.tensor([1, 2, 3, 4, 5, 6, 7])
print(a.storage().data_ptr())

a = a.resize_(2, 3)
print(a)
print(a.storage())
print(a.storage().data_ptr())

140203173778560
tensor([[1, 2, 3],
        [4, 5, 6]])
 1
 2
 3
 4
 5
 6
 7
[torch.storage.TypedStorage(dtype=torch.int64, device=cpu) of size 7]
140203173778560


### 2）当原始数据 元素不够的时候
如果原始数据不够，它会开辟一个新的存储区，并用0进行填充，凑满你要的尺寸

In [50]:
import torch

a = torch.tensor([1, 2, 3, 4, 5])
print(a.storage().data_ptr())

a = a.resize_(2, 3)
print(a)
print(a.storage())
print(a.storage().data_ptr())

140203159189120
tensor([[1, 2, 3],
        [4, 5, 0]])
 1
 2
 3
 4
 5
 0
[torch.storage.TypedStorage(dtype=torch.int64, device=cpu) of size 6]
140203170383296
